In [ ]:
from src import data
import pandas as pd
import math
import numpy as np
from tqdm import tqdm

In [ ]:
speed = data.speeds('train')
bsm = pd.read_csv('/home/giovanni/Desktop/DMProject/resources/dataset/preprocessed/base_structure_df_weather.csv.gz')
bsm = bsm.drop(['Unnamed: 0'], axis=1)
bsm.DATETIME_UTC = pd.to_datetime(bsm.DATETIME_UTC)
bsm['hour'] = pd.to_datetime(bsm.DATETIME_UTC).dt.hour

In [ ]:
bsm.WEATHER.unique()

In [ ]:
bsm

In [ ]:
d = {}
bsm_dropped_duplicates = bsm.drop_duplicates(subset=['KEY', 'KM'])[['KEY', 'KM']]
keys = bsm_dropped_duplicates.KEY.values
kms = bsm_dropped_duplicates.KM.values
for idx in tqdm(range(len(kms))):
    key = keys[idx]
    km = kms[idx]
    fs = bsm[(bsm.KEY == key) & (bsm.KM == km)]
    speeds = speed[(speed.KEY == key) & (speed.KM == km)]
    fmerged = speeds.merge(fs)
    for h in range(24):
        merged = fmerged[(fmerged.hour == h)]        
        mean_street = merged.SPEED_AVG.mean()
        for wt in merged.WEATHER.unique():
            slicew = merged[merged.WEATHER == wt]
            slice_mean = slicew.SPEED_AVG.mean()
            if type(wt) != float:
                if wt not in d:
                    d[wt] = [mean_street-slice_mean]
                else:
                    d[wt].append(mean_street-slice_mean)
    #print('weather condition {}. mean: {}. diff: {}'.format(wt, slice_mean, abs(mean_street-slice_mean)))
#d

In [ ]:
d_mean = {}
for k,v in d.items():
    d_mean[k] = (-sum(v)/len(v), len(v))
d_mean

In [ ]:
len(d)

In [ ]:
d_mean = {}
for k,v in d.items():
    d_mean[k] = -sum(v)/len(v)
d_mean

In [ ]:
clst_1=['Forte Pioggerella','Temporale con Forte  Neve Pioggia e  Grandine']
clst_2=['Temporale con Debole Neve Pioggia e Grandine','Temporale con Grandine']
clst_3=['Forte Pioggia','Rovescio con Forte Pioggia']
clst_4=['Debole Pioggia e Pioggrella','Temporale con Debole Pioggia','Rovescio con Grandine Piccola','Rovescio con Debole Pioggia','Debole Pioggia']
clst_5=['Debole Pioggia e Neve','Debole Neve']
clst_6=['Tempesta di Polvere','Nelle Vicinanze Tempesta di Polvere']
clst_7=['Debole Neve a Granuli','Neve e Pioggia']
clst_8=['Nelle Vicinanze Nebbia','Banchi di Nebbia','Sottili Banchi di Nebbia','Foschia']

clusters=[clst_1,clst_2,clst_3,clst_4,clst_5,clst_6,clst_7,clst_8]
clusters_names=['Forte Pioggerella','Temporale con Debole Neve Pioggia e Grandine','Forte Pioggia','Debole Pioggia e Pioggrella','Debole Pioggia e Neve','Tempesta di Polvere','Debole Neve a Granuli','Nelle Vicinanze Nebbia']

In [ ]:
bsm["WEATHER_CL"]=""
it=0
for cl in tqdm(clusters):
    for c in tqdm(cl):
        for row in bsm.loc[bsm["WEATHER"]==c].iterrows():
             bsm.at[row[0],'WEATHER_CL'] = clusters_names[it]
            
        
    it = it+1
        

In [ ]:
for row in bsm.loc[bsm["WEATHER_CL"]==""].iterrows():
    bsm.at[row[0],'WEATHER_CL'] = row.WEATHER